In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
sys.path.insert(0,"..")
sys.path.insert(0, "/workspace/update/torchxrayvision")
sys.path.insert(0, "/workspace/update/torchxrayvision/torchxrayvision")
from glob import glob
from os.path import exists, join
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import argparse

import torch
import torchvision, torchvision.transforms
import skimage.transform
import sklearn

import random
import train_utils
import pickle
import torchxrayvision as xrv


In [4]:
! pwd

/workspace/update/torchxrayvision/scripts/calibration


In [3]:
!ls res*

results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped-best_chex-nih-mimic_ch-pc-google-openi-kaggle-mimic_nb.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped-best_chex-nih-mimic_ch-pc-google-openi-kaggle.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped-best_chex-nih_relabel-mimic_ch-pc-google-openi-kaggle-mimic_nb.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped-best_chex-nih_relabel-mimic_ch-pc-google-openi-kaggle.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped_relabel-nih-best_chex-nih-mimic_ch-pc-google-openi-kaggle-mimic_nb.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped_relabel-nih-best_chex-nih-mimic_ch-pc-google-openi-kaggle.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped_relabel-nih-best_chex-nih_relabel-mimic_ch-pc-google-openi-kaggle-mimic_nb.pkl
results_chex-nih-mimic_ch-pc-google-openi-kaggle-densenet121-cropped_re

In [131]:
base_path = "/workspace/update/torchxrayvision/scripts/calibration/"
pkl_name = "results_chex-nih-mimic_nb-mimic_ch-pc-google-openi-kaggle-densenet121-model_2_cropped-best_chex-nih-mimic_ch-pc-google-openi-kaggle-mimic_nb.pkl"

In [132]:
import pickle
path_pkl = base_path + pkl_name
with open(path_pkl, 'rb') as f:
    results = pickle.load(f)


In [152]:
results[3]

{0: array([0., 0., 0., ..., 0., 0., 0.]),
 1: array([0., 0., 0., ..., 0., 0., 0.]),
 2: array([0., 0., 0., ..., 0., 0., 0.]),
 3: array([0., 0., 0., ..., 0., 0., 0.]),
 4: array([0., 0., 0., ..., 0., 0., 0.]),
 5: array([0., 0., 0., ..., 0., 0., 0.]),
 6: array([0., 0., 1., ..., 0., 0., 0.]),
 7: array([0., 0., 0., ..., 0., 0., 0.]),
 8: array([0., 0., 0., ..., 0., 0., 1.]),
 9: array([0., 0., 0., ..., 0., 0., 0.]),
 10: array([0., 0., 0., ..., 0., 0., 0.]),
 11: array([0., 0., 0., ..., 0., 0., 0.]),
 12: array([0., 0., 0., ..., 0., 0., 0.]),
 13: array([0., 0., 0., ..., 0., 0., 0.]),
 14: array([0., 0., 0., ..., 0., 0., 0.]),
 15: array([0., 0., 0., ..., 0., 0., 0.]),
 16: array([0., 0., 0., ..., 0., 0., 1.]),
 17: array([0., 0., 0., ..., 0., 0., 0.])}

In [154]:
results[2][1] 

(array([-7.114351 , -5.7151794, -2.8105702, ..., -4.2162313, -5.1159444,
        -6.953275 ], dtype=float32),
 84163)

In [155]:
len(results[2][1])

84163

In [135]:
all_threshs = []
all_min = []
all_max = []
all_ppv80 = []
for i, patho in enumerate(xrv.datasets.default_pathologies):
    opt_thres = np.nan
    opt_max = np.nan
    if len(results[3][i]) > 0:
        
        #sigmoid
        all_outputs = 1.0/(1.0 + np.exp(-results[2][i]))
        
        fpr, tpr, thres = sklearn.metrics.roc_curve(results[3][i], all_outputs)
        pente = tpr - fpr
        opt_thres = thres[np.argmax(pente)]
        opt_min = all_outputs.min()
        opt_max = all_outputs.max()
        
        ppv, recall, thres = sklearn.metrics.precision_recall_curve(results[3][i], all_outputs)
        ppv80_thres_idx = np.where(ppv > 0.8)[0][0]
        ppv80_thres = thres[ppv80_thres_idx-1]
        
        
    all_threshs.append(opt_thres)
    all_min.append(opt_min)
    all_max.append(opt_max)
    all_ppv80.append(ppv80_thres)

In [136]:
print(str(all_threshs).replace("nan","np.nan"))

[0.08653892, 0.017242203, 0.061408978, 0.006965817, 0.007230344, 0.008352524, 0.0052154628, 0.09193552, 0.035084374, 0.020144384, 0.0809918, 0.032811552, 0.0071497215, 0.014541325, 0.019512547, 0.031239916, 0.11984462, 0.00936549]


In [137]:
print(str(all_min).replace("nan","np.nan"))

[7.2611583e-06, 3.8029718e-06, 0.00043354367, 4.167906e-07, 4.7786166e-09, 1.189347e-08, 9.09408e-09, 1.489974e-05, 2.1840038e-05, 1.8500214e-06, 2.5899712e-06, 0.00024218779, 1.956632e-07, 1.0862463e-10, 1.6243986e-08, 5.507509e-05, 0.000112184236, 1.574199e-08]


In [138]:
print(str(all_max).replace("nan","np.nan"))

[0.9997209, 0.9989944, 0.57375365, 0.9377187, 0.99945325, 0.8887256, 0.4997312, 0.9995877, 0.9962728, 0.8359465, 0.9961461, 0.9338063, 0.9997881, 0.8658175, 0.9999949, 0.99887985, 0.99982363, 0.999757]


In [139]:
print(str(all_ppv80).replace("nan","np.nan"))

[0.9937518, 0.99484503, 0.57375365, 0.9282644, 0.9529527, 0.5661315, 0.4997312, 0.91090864, 0.99205655, 0.8359465, 0.9208461, 0.91701555, 0.9997881, 0.8658175, 0.99993837, 0.99887985, 0.8762074, 0.9988675]


In [140]:
ppv, recall, thres = sklearn.metrics.precision_recall_curve(results[3][0], results[2][0], pos_label=1)

In [141]:
ppv

array([0.08684627, 0.08682502, 0.08682704, ..., 1.        , 1.        ,
       1.        ])

In [142]:
recall, len(recall)

(array([1.00000000e+00, 9.99732119e-01, 9.99732119e-01, ...,
        2.67881061e-04, 1.33940530e-04, 0.00000000e+00]),
 51416)

In [143]:
thres, len(thres)

(array([-8.1549   , -8.153434 , -8.152814 , ...,  7.6707125,  7.7086   ,
         8.183672 ], dtype=float32),
 51415)

In [144]:
ppv80_thres = np.where(ppv > 0.8)[0][0]

In [145]:
ppv80_thres = np.where(npv > 0.8)[0][0]

NameError: name 'npv' is not defined

In [146]:
thres[ppv80_thres]

5.0772295

In [147]:
model_urls = {}
model_urls['all'] = {
    "weights_url":
    'https://github.com/mlmed/torchxrayvision/releases/download/v1/nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best.pt',
    "op_threshs": [
        0.07422872, 0.038290843, 0.09814756, 0.0098118475, 0.023601074,
        0.0022490358, 0.010060724, 0.103246614, 0.056810737, 0.026791653,
        0.050318155, 0.023985857, 0.01939503, 0.042889766, 0.053369623,
        0.035975814, 0.20204692, 0.05015312
    ],
    "ppv80_thres": [
        0.72715247, 0.8885005, 0.92493945, 0.6527224, 0.68707734, 0.46127197,
        0.7272054, 0.6127343, 0.9878492, 0.61979693, 0.66309816, 0.7853459,
        0.930661, 0.93645346, 0.6788558, 0.6547198, 0.61614525, 0.8489876
    ],
    "base":
    "cohen",
    "area_opacity": {
        "theta": 0.8705248236656189,
        "bias": 3.4137437
    },
    "degree_opacity": {
        "theta": 0.5484423041343689,
        "bias": 2.5535977
    }
}

In [150]:
type(model_urls["all"]["area_opacity"]["theta"])

float

In [149]:
model_urls["all"]["area_opacity"]["bias"]

3.4137437